In [ ]:
#In this notebook we scrape vegas odds for upcoming fights and save the data to vegas_odds.json for use on website

In [1]:
#needed after restructuring of directory
import os
os.chdir('/Users/alexchandler/UFC_Prediction_2022')

#!/usr/bin/env python
# coding: utf-8

# In[ ]:

import requests
import urllib3
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

#turning csv files to json files
import csv
import json

 
# Function to convert a CSV to JSON
# Takes the file paths as arguments
def make_json(csvFilePath, jsonFilePath, column):
     
    # create a dictionary
    data = {}
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:
             
            # primary key given by column variable
            key = rows[column]
            data[key] = rows
 
    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
        
def get_odds_two_rows_per_fight():
    url = 'https://www.bestfightodds.com'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser") 
    mydivs = soup.find_all("tr", {"class": ""})
    rows=[tr for tr in mydivs if 'bestbet' in str(tr)]
    names=[]
    oddsDicts=[]
    books=['DraftKings','BetMGM','Caesars','BetRivers','FanDuel','PointsBet','Unibet','Bet365','BetWay','5D','Ref']
    for row in rows:
        #gets name of fighter in row
        name = row.find_all("span", {"class": "t-b-fcc"})[0].text
        oddsList=[name]
        i=0
        for stat in row.select('td'):
            i+=1
            if i>11:
                break
            try:
                odds = stat.select('span')[0].text
                oddsList.append(odds)
            except:
                oddsList.append('')
        names.append(name)
        oddsDicts.append(dict(zip(['name']+books,oddsList)))
    oddsDict = dict(zip(names,oddsDicts))
    names = list(oddsDict.keys())
    row0=oddsDict[list(oddsDict.keys())[0]]
    odds_df = pd.DataFrame(row0, index=[0])
    for i in range(1,len(names)):
        row=oddsDict[names[i]]
        odds_df = pd.concat([odds_df, pd.DataFrame(row, index=[i])], axis = 0)
    return odds_df


#problem: the fights in the "future events" category do not get lined up properly
def get_odds():
    url = 'https://www.bestfightodds.com'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser") 
    mydivs = soup.find_all("tr", {"class": ""})
    rows=[tr for tr in mydivs if 'bestbet' in str(tr)]
    names=[]
    oddsDicts=[]
    books=['DraftKings','BetMGM','Caesars','BetRivers','FanDuel','PointsBet','Unibet','Bet365','BetWay','5D','Ref']
    for row in rows:
        #gets name of fighter in row
        name = row.find_all("span", {"class": "t-b-fcc"})[0].text
        oddsList=[name]
        i=0
        for stat in row.select('td'):
            i+=1
            if i>11:
                break
            try:
                odds = stat.select('span')[0].text
                oddsList.append(odds)
            except:
                oddsList.append('')
        if name in names:
            names.append(name+'.')
        else:
            names.append(name)
        oddsDicts.append(dict(zip(['name']+books,oddsList)))
    oddsDict = dict(zip(names,oddsDicts))
    names = list(oddsDict.keys())
    row0=oddsDict[list(oddsDict.keys())[0]]
    odds_df = pd.DataFrame(row0, index=[0])
    for i in range(1,len(names)):
        row=oddsDict[names[i]]
        odds_df = pd.concat([odds_df, pd.DataFrame(row, index=[i])], axis = 0)
    #making it so each fight has just a single row instead of two rows
    #making dataframe just for even indexed columns
    odds_df_evens = odds_df[odds_df.index%2==0]
    newcolumns1={}
    for col in list(odds_df_evens.columns):
        newcolumns1[col]='fighter '+col
    odds_df_evens=odds_df_evens.rename(columns=newcolumns1)
    odds_df_evens.reset_index(drop=True, inplace=True)
    #making dataframe just for odd indexed columns
    odds_df_odds = odds_df[odds_df.index%2==1]
    newcolumns2={}
    for col in list(odds_df_odds.columns):
        newcolumns2[col]='opponent '+col
    odds_df_odds=odds_df_odds.rename(columns=newcolumns2)
    odds_df_odds.reset_index(drop=True, inplace=True)
    new_odds_df = pd.concat([odds_df_evens, odds_df_odds], axis = 1)
    return new_odds_df


In [2]:
#scrapes odds from bestfightodds.com
odds_df = get_odds();odds_df

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref
0,Ion Curelaba,-210,-225,-240,-225,-225,,-225,-227,-225,-220,-221,Ryan Spann,+175,+185,+200,+180,+188,,+180,+187,+175,+200,+186
1,Davey Grant,-280,-300,-280,-305,-300,,-305,-278,-333,-290,-280,Louis Smolka,+225,+240,+230,+225,+245,,+225,+225,+250,+260,+240
2,Amanda Ribas,+140,+150,+135,+136,+144,,+136,+162,+150,+160,+150,Katlyn Chookagian,-160,-185,-160,-175,-172,,-175,-189,-188,-170,-175
3,Frank Camacho,+110,+105,+105,+105,+112,,+105,+110,+100,+115,+110,Manuel Torres,-130,-125,-125,-136,-132,,-136,-139,-125,-125,-130
4,Allan Nascimento,+175,+185,+180,+175,+188,,+175,+180,+175,+190,+179,Jake Hadley,-210,-225,-220,-225,-225,,-225,-222,-225,-210,-209
5,Andrea Lee,-125,-120,-120,-122,-116,,-122,-125,-125,-115,-124,Viviane Araujo,+105,+100,+100,-104,-102,,-104,+100,+100,+105,+104
6,Alan Patrick,+125,+130,+125,+120,+122,,+120,+125,+120,+132,+126,Michael Johnson,-145,-155,-150,-150,-144,,-150,-152,-150,-142,-146
7,Angela Hill,+170,+150,+160,+150,+154,,+150,+162,+150,+170,+172,Virna Jandiroba,-200,-185,-190,-190,-184,,-190,-189,-188,-185,-202
8,Carlos Candelario,+230,+230,+250,+225,+210,,+225,+200,+200,+265,+255,Tatsuro Taira,-290,-300,-310,-295,-255,,-295,-250,-250,-295,-305
9,Andre Petroski,+280,+280,+300,+290,+300,,+290,+275,+300,+295,+285,Nick Maximov,-365,-375,-380,-400,-375,,-400,-357,-400,-325,-350


In [3]:
#thresh is the number of bookies we allow to not have odds on the books
def drop_irrelevant_fights(df,thresh):
    irr=[]
    for i in df.index:
        count=0
        row=list(df.loc[i])
        for j in row:
            if j=='':
                count+=1
        if count>2*thresh:
            irr.append(i)
    df=df.drop(irr)
    return df

In [4]:
odds_df=drop_irrelevant_fights(odds_df,1)

In [5]:
odds_df

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref
0,Ion Curelaba,-210,-225,-240,-225,-225,,-225,-227,-225,-220,-221,Ryan Spann,+175,+185,+200,+180,+188,,+180,+187,+175,+200,+186
1,Davey Grant,-280,-300,-280,-305,-300,,-305,-278,-333,-290,-280,Louis Smolka,+225,+240,+230,+225,+245,,+225,+225,+250,+260,+240
2,Amanda Ribas,+140,+150,+135,+136,+144,,+136,+162,+150,+160,+150,Katlyn Chookagian,-160,-185,-160,-175,-172,,-175,-189,-188,-170,-175
3,Frank Camacho,+110,+105,+105,+105,+112,,+105,+110,+100,+115,+110,Manuel Torres,-130,-125,-125,-136,-132,,-136,-139,-125,-125,-130
4,Allan Nascimento,+175,+185,+180,+175,+188,,+175,+180,+175,+190,+179,Jake Hadley,-210,-225,-220,-225,-225,,-225,-222,-225,-210,-209
5,Andrea Lee,-125,-120,-120,-122,-116,,-122,-125,-125,-115,-124,Viviane Araujo,+105,+100,+100,-104,-102,,-104,+100,+100,+105,+104
6,Alan Patrick,+125,+130,+125,+120,+122,,+120,+125,+120,+132,+126,Michael Johnson,-145,-155,-150,-150,-144,,-150,-152,-150,-142,-146
7,Angela Hill,+170,+150,+160,+150,+154,,+150,+162,+150,+170,+172,Virna Jandiroba,-200,-185,-190,-190,-184,,-190,-189,-188,-185,-202
8,Carlos Candelario,+230,+230,+250,+225,+210,,+225,+200,+200,+265,+255,Tatsuro Taira,-290,-300,-310,-295,-255,,-295,-250,-250,-295,-305
9,Andre Petroski,+280,+280,+300,+290,+300,,+290,+275,+300,+295,+285,Nick Maximov,-365,-375,-380,-400,-375,,-400,-357,-400,-325,-350


In [6]:
good_indices = [0,1,2,3,4,5,6,7,8,9,77]

In [7]:
odds_df=odds_df.loc[good_indices];odds_df

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref
0,Ion Curelaba,-210,-225,-240,-225,-225,,-225,-227,-225,-220,-221,Ryan Spann,+175,+185,+200,+180,+188,,+180,+187,+175,+200,+186
1,Davey Grant,-280,-300,-280,-305,-300,,-305,-278,-333,-290,-280,Louis Smolka,+225,+240,+230,+225,+245,,+225,+225,+250,+260,+240
2,Amanda Ribas,+140,+150,+135,+136,+144,,+136,+162,+150,+160,+150,Katlyn Chookagian,-160,-185,-160,-175,-172,,-175,-189,-188,-170,-175
3,Frank Camacho,+110,+105,+105,+105,+112,,+105,+110,+100,+115,+110,Manuel Torres,-130,-125,-125,-136,-132,,-136,-139,-125,-125,-130
4,Allan Nascimento,+175,+185,+180,+175,+188,,+175,+180,+175,+190,+179,Jake Hadley,-210,-225,-220,-225,-225,,-225,-222,-225,-210,-209
5,Andrea Lee,-125,-120,-120,-122,-116,,-122,-125,-125,-115,-124,Viviane Araujo,+105,+100,+100,-104,-102,,-104,+100,+100,+105,+104
6,Alan Patrick,+125,+130,+125,+120,+122,,+120,+125,+120,+132,+126,Michael Johnson,-145,-155,-150,-150,-144,,-150,-152,-150,-142,-146
7,Angela Hill,+170,+150,+160,+150,+154,,+150,+162,+150,+170,+172,Virna Jandiroba,-200,-185,-190,-190,-184,,-190,-189,-188,-185,-202
8,Carlos Candelario,+230,+230,+250,+225,+210,,+225,+200,+200,+265,+255,Tatsuro Taira,-290,-300,-310,-295,-255,,-295,-250,-250,-295,-305
9,Andre Petroski,+280,+280,+300,+290,+300,,+290,+275,+300,+295,+285,Nick Maximov,-365,-375,-380,-400,-375,,-400,-357,-400,-325,-350


In [9]:
n=77
row = odds_df.loc[n].to_frame()
df = odds_df.drop([n])
row.index
new_row = pd.DataFrame(dict(zip(row.index,list(row[n]))), index=[0])

In [10]:
new_row

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref
0,Aleksandar Rakic,-200,-200,-220,-200,-198,,-200,-189,-200,-200,-200,Jan Blachowicz,+170,+165,+180,+155,+166,,+155,+162,+163,+185,+170


In [11]:
odds_df = pd.concat([new_row, df]).reset_index(drop = True)

In [12]:
odds_df

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref
0,Aleksandar Rakic,-200,-200,-220,-200,-198,,-200,-189,-200,-200,-200,Jan Blachowicz,+170,+165,+180,+155,+166,,+155,+162,+163,+185,+170
1,Ion Curelaba,-210,-225,-240,-225,-225,,-225,-227,-225,-220,-221,Ryan Spann,+175,+185,+200,+180,+188,,+180,+187,+175,+200,+186
2,Davey Grant,-280,-300,-280,-305,-300,,-305,-278,-333,-290,-280,Louis Smolka,+225,+240,+230,+225,+245,,+225,+225,+250,+260,+240
3,Amanda Ribas,+140,+150,+135,+136,+144,,+136,+162,+150,+160,+150,Katlyn Chookagian,-160,-185,-160,-175,-172,,-175,-189,-188,-170,-175
4,Frank Camacho,+110,+105,+105,+105,+112,,+105,+110,+100,+115,+110,Manuel Torres,-130,-125,-125,-136,-132,,-136,-139,-125,-125,-130
5,Allan Nascimento,+175,+185,+180,+175,+188,,+175,+180,+175,+190,+179,Jake Hadley,-210,-225,-220,-225,-225,,-225,-222,-225,-210,-209
6,Andrea Lee,-125,-120,-120,-122,-116,,-122,-125,-125,-115,-124,Viviane Araujo,+105,+100,+100,-104,-102,,-104,+100,+100,+105,+104
7,Alan Patrick,+125,+130,+125,+120,+122,,+120,+125,+120,+132,+126,Michael Johnson,-145,-155,-150,-150,-144,,-150,-152,-150,-142,-146
8,Angela Hill,+170,+150,+160,+150,+154,,+150,+162,+150,+170,+172,Virna Jandiroba,-200,-185,-190,-190,-184,,-190,-189,-188,-185,-202
9,Carlos Candelario,+230,+230,+250,+225,+210,,+225,+200,+200,+265,+255,Tatsuro Taira,-290,-300,-310,-295,-255,,-295,-250,-250,-295,-305


In [13]:
#saves the scraped odds to the file vegas_odds.json in src/models/buildingMLModel/data/external/
result = odds_df.to_json()
parsed = json.loads(result)
jsonFilePath='src/models/buildingMLModel/data/external/vegas_odds.json'
with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
    jsonf.write(json.dumps(parsed, indent=4))
print('saved to '+jsonFilePath)

saved to src/models/buildingMLModel/data/external/vegas_odds.json


In [43]:
#run this if you want to see what the dataframe looks like before row doubling
odds_df_two=get_odds_two_rows_per_fight();odds_df_two

,name,DraftKings,BetMGM,Caesars,BetRivers,FanDuel,PointsBet,Unibet,Bet365,BetWay,5D,Ref
0,Dejan Kajic,,,,,,,,,,,+155
1,Saeid Mirzaei,,,,,,,,,,,-185
2,Adam de Freitas,,,,,,,,,,,+160
3,Christian Tremayne,,,,,,,,,,,-200
4,Ozhan Yalcin,,,,,,,,,,,-700
5,Wesley Bowman,,,,,,,,,,,+450
6,Austin Russell,,,,,,,,,,,+160
7,Serhiy Sidey,,,,,,,,,,,-200
8,James Foster,,,,,,,,,,,+135
9,Rafael Palomeque,,,,,,,,,,,-165


In [ ]:
#use this if you need to add custom rows to vegas_odds

In [5]:
# if you want to add a new row from bestfightodds.com, copy and paste the two rows, 
# find and replace the arrows with commas, delete extra commas, turn names into strings, 
new_row_data = ['Charles Oliveira',-170,-165,-170,-177,-172,-165,-177,-164,-175,-170,-170,
'Justin Gaethje',+150,+140,+145,+140,+140,+136,+140,+137,+140,+150,+145]

#make data into dataframe column with correct column names and add to odds_df
new_row_data = dict(zip(odds_df.columns,new_row_data))
new_row = pd.DataFrame(new_row_data, index =[0])
odds_df = pd.concat([new_row, odds_df]).reset_index(drop = True);odds_df

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref
0,Charles Oliveira,-170,-165,-170,-177,-172,-165,-177,-164,-175,-170,-170,Justin Gaethje,150,140,145,140,140,136,140,137,140,150,145
1,Carla Esparza,+175,+185,+180,+158,+168,+155,+158,+175,+175,+180,+175,Rose Namajunas,-210,-225,-220,-210,-200,-190,-210,-213,-225,-220,-205
2,Michael Chandler,-365,-400,-360,-400,-360,-400,-400,-400,-400,-345,-400,Tony Ferguson,+280,+300,+280,+300,+290,+300,+300,+300,+300,+285,+300
3,Mauricio Rua,+195,+200,+190,+200,+205,+210,+200,+200,+200,+205,+210,Ovince St. Preux,-240,-250,-230,-265,-250,-260,-265,-250,-250,-245,-250
4,Khaos Williams,-125,-120,-120,-125,-120,-135,-125,-125,-120,-120,-120,Randy Brown,+105,+100,+100,+101,+102,+112,+101,+100,-105,+100,+100
5,Macy Chiasson,+185,+180,+180,+183,+198,+195,+183,+180,+175,+185,+185,Norma Dumont,-225,-225,-220,-235,-240,-240,-235,-222,-225,-225,-220
6,Brandon Royval,-240,-275,-260,-265,-255,-270,-265,-250,-250,-245,-220,Matt Schnell,+195,+220,+210,+205,+210,+215,+205,+200,+200,+205,+185
7,Blagoy Ivanov,-145,-145,-155,-152,-156,-150,-152,-152,-150,-150,-151,Marcos Rogerio,+125,+120,+130,+122,+132,+125,+122,+125,+120,+130,+131
8,Danny Roberts,-105,-110,-105,-103,-104,+100,-103,+100,-105,-105,-110,Francisco Trinaldo,-115,-110,-115,-121,-112,-120,-121,-125,-120,-115,-110
9,Melissa Gatto,+125,+120,+120,+128,+128,+150,+128,+130,+120,+130,+120,Tracy Cortez,-145,-145,-140,-159,-152,-180,-159,-161,-150,-150,-140


In [6]:
#if it looks ok, save it
#IMPORTANT. NEXT GO TO UFC_Prediction_Model notebook and run predictions after exporting vegas_odds.json
#saves the scraped odds to the file vegas_odds.json in src/models/buildingMLModel/data/external/
result = odds_df.to_json()
parsed = json.loads(result)
jsonFilePath='src/models/buildingMLModel/data/external/vegas_odds.json'
with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
    jsonf.write(json.dumps(parsed, indent=4))
print('saved to '+jsonFilePath)

saved to src/models/buildingMLModel/data/external/vegas_odds.json


In [ ]:
# BELOW WE MANUALLY FIX PREDICTION HISTORY IF IT EVER GETS MESSED UP
# IF YOU DON'T NEED TO FIX PREDICTION_HISTORY, DO NOT RUN THE CELLS BELOW

In [14]:
#sometimes prediction_history gets messed up... here we can fix it manually
prediction_history = pd.read_json('src/models/buildingMLModel/data/external/prediction_history.json');prediction_history

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref,predicted fighter odds,predicted opponent odds,average bookie odds,correct?
0,Donald Cerrone,-195,-185,-200,-200,-188,-195,-200,-189,-188,-200,-210,Joe Lauzon,165,150,170,155,158,160,155,162,155,170,180,-120,+120,"[-195, 162]",
1,Glover Teixeira,180,165,165,152,160,160,152,170,163,165,165,Jiri Prochazka,-220,-200,-195,-200,-190,-195,-200,-200,-200,-190,-190,-114,+114,"[163, -198]",
2,Taila Santos,550,625,550,530,570,550,530,550,500,550,500,Valentina Shevchenko,-800,-1000,-800,-835,-820,-800,-835,-833,-800,-800,-800,+126,-126,"[546, -829]",
3,Charles Oliveira,-130,-145,-150,-148,-144,-165,-148,-139,-138,-135,-145,Justin Gaethje,110,120,125,118,122,136,118,110,110,115,125,-447,+447,"[-144, 119]",
4,Gilbert Burns,375,400,400,370,385,380,370,450,400,400,375,Arman Tsarukyan,-265,-350,-280,-250,-295,-260,-250,-333,-250,-290,-315,+307,-307,"[391, -285]",
5,Joel Alvarez,215,280,230,195,240,200,195,250,200,245,265,Calvin Cattar,185,200,190,195,215,180,195,200,175,190,205,-176,+176,"[229, 194]",
12,Carla Esparza,175,185,180,158,168,155,158,175,175,180,175,Rose Namajunas,-210,-225,-220,-210,-200,-190,-210,-213,-225,-220,-205,-134,+134,"[171, -212]",
13,Michael Chandler,-365,-400,-360,-400,-360,-400,-400,-400,-400,-345,-400,Tony Ferguson,280,300,280,300,290,300,300,300,300,285,300,+127,-127,"[-385, 294]",
14,Mauricio Rua,195,200,190,200,205,210,200,200,200,205,210,Ovince St. Preux,-240,-250,-230,-265,-250,-260,-265,-250,-250,-245,-250,+201,-201,"[201, -250]",
15,Khaos Williams,-125,-120,-120,-125,-120,-135,-125,-125,-120,-120,-120,Randy Brown,105,100,100,101,102,112,101,100,-105,100,100,-186,+186,"[-123, 83]",


In [15]:
#found some rows that should not be there and removing by hand (need to fix python scripts to avoid having to do this)
for i in [0,1,2,4,5]:
    prediction_history=prediction_history.drop([i])
prediction_history

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref,predicted fighter odds,predicted opponent odds,average bookie odds,correct?
3,Charles Oliveira,-130,-145,-150,-148,-144,-165,-148,-139,-138,-135,-145,Justin Gaethje,110,120,125,118,122,136,118,110,110,115,125,-447,+447,"[-144, 119]",
12,Carla Esparza,175,185,180,158,168,155,158,175,175,180,175,Rose Namajunas,-210,-225,-220,-210,-200,-190,-210,-213,-225,-220,-205,-134,+134,"[171, -212]",
13,Michael Chandler,-365,-400,-360,-400,-360,-400,-400,-400,-400,-345,-400,Tony Ferguson,280,300,280,300,290,300,300,300,300,285,300,+127,-127,"[-385, 294]",
14,Mauricio Rua,195,200,190,200,205,210,200,200,200,205,210,Ovince St. Preux,-240,-250,-230,-265,-250,-260,-265,-250,-250,-245,-250,+201,-201,"[201, -250]",
15,Khaos Williams,-125,-120,-120,-125,-120,-135,-125,-125,-120,-120,-120,Randy Brown,105,100,100,101,102,112,101,100,-105,100,100,-186,+186,"[-123, 83]",
16,Macy Chiasson,185,180,180,183,198,195,183,180,175,185,185,Norma Dumont,-225,-225,-220,-235,-240,-240,-235,-222,-225,-225,-220,-112,+112,"[184, -228]",
17,Brandon Royval,-240,-275,-260,-265,-255,-270,-265,-250,-250,-245,-220,Matt Schnell,195,220,210,205,210,215,205,200,200,205,185,-124,+124,"[-254, 205]",
18,Blagoy Ivanov,-145,-145,-155,-152,-156,-150,-152,-152,-150,-150,-151,Marcos Rogerio de lima,125,120,130,122,132,125,122,125,120,130,131,+103,-103,"[-151, 126]",
19,Danny Roberts,-105,-110,-105,-103,-104,100,-103,100,-105,-105,-110,Francisco Trinaldo,-115,-110,-115,-121,-112,-120,-121,-125,-120,-115,-110,-101,+101,"[-68, -117]",
20,Melissa Gatto,125,120,120,128,128,150,128,130,120,130,120,Tracy Cortez,-145,-145,-140,-159,-152,-180,-159,-161,-150,-150,-140,-104,+104,"[127, -153]",


In [17]:
result = prediction_history.to_json()
parsed = json.loads(result)
jsonFilePath='src/models/buildingMLModel/data/external/prediction_history.json'
with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
    jsonf.write(json.dumps(parsed, indent=4))
print('saved to '+jsonFilePath)

saved to src/models/buildingMLModel/data/external/prediction_history.json


In [15]:
prediction_history

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref,predicted fighter odds,predicted opponent odds,average bookie odds,correct?
0,Charles Oliveira,-170,-165,-170,-177,-172,-165,-177,-164,-175,-170,-170,Justin Gaethje,150,140,145,140,140,136,140,137,140,150,145,-540,+540,"[-170, 142]",
1,Carla Esparza,175,185,180,158,168,155,158,175,175,180,175,Rose Namajunas,-210,-225,-220,-210,-200,-190,-210,-213,-225,-220,-205,-134,+134,"[171, -212]",
2,Michael Chandler,-365,-400,-360,-400,-360,-400,-400,-400,-400,-345,-400,Tony Ferguson,280,300,280,300,290,300,300,300,300,285,300,+127,-127,"[-385, 294]",
3,Mauricio Rua,195,200,190,200,205,210,200,200,200,205,210,Ovince St. Preux,-240,-250,-230,-265,-250,-260,-265,-250,-250,-245,-250,+201,-201,"[201, -250]",
4,Khaos Williams,-125,-120,-120,-125,-120,-135,-125,-125,-120,-120,-120,Randy Brown,105,100,100,101,102,112,101,100,-105,100,100,-186,+186,"[-123, 83]",
5,Macy Chiasson,185,180,180,183,198,195,183,180,175,185,185,Norma Dumont,-225,-225,-220,-235,-240,-240,-235,-222,-225,-225,-220,-112,+112,"[184, -228]",
6,Brandon Royval,-240,-275,-260,-265,-255,-270,-265,-250,-250,-245,-220,Matt Schnell,195,220,210,205,210,215,205,200,200,205,185,-124,+124,"[-254, 205]",
7,Blagoy Ivanov,-145,-145,-155,-152,-156,-150,-152,-152,-150,-150,-151,Marcos Rogerio,125,120,130,122,132,125,122,125,120,130,131,,,,
8,Danny Roberts,-105,-110,-105,-103,-104,100,-103,100,-105,-105,-110,Francisco Trinaldo,-115,-110,-115,-121,-112,-120,-121,-125,-120,-115,-110,-101,+101,"[-68, -117]",
9,Melissa Gatto,125,120,120,128,128,150,128,130,120,130,120,Tracy Cortez,-145,-145,-140,-159,-152,-180,-159,-161,-150,-150,-140,-104,+104,"[127, -153]",


In [16]:
#use this to fix any names in the dataframe
prediction_history.at[7, 'opponent name'] = 'Marcos Rogerio de lima'
prediction_history

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref,predicted fighter odds,predicted opponent odds,average bookie odds,correct?
0,Charles Oliveira,-170,-165,-170,-177,-172,-165,-177,-164,-175,-170,-170,Justin Gaethje,150,140,145,140,140,136,140,137,140,150,145,-540,+540,"[-170, 142]",
1,Carla Esparza,175,185,180,158,168,155,158,175,175,180,175,Rose Namajunas,-210,-225,-220,-210,-200,-190,-210,-213,-225,-220,-205,-134,+134,"[171, -212]",
2,Michael Chandler,-365,-400,-360,-400,-360,-400,-400,-400,-400,-345,-400,Tony Ferguson,280,300,280,300,290,300,300,300,300,285,300,+127,-127,"[-385, 294]",
3,Mauricio Rua,195,200,190,200,205,210,200,200,200,205,210,Ovince St. Preux,-240,-250,-230,-265,-250,-260,-265,-250,-250,-245,-250,+201,-201,"[201, -250]",
4,Khaos Williams,-125,-120,-120,-125,-120,-135,-125,-125,-120,-120,-120,Randy Brown,105,100,100,101,102,112,101,100,-105,100,100,-186,+186,"[-123, 83]",
5,Macy Chiasson,185,180,180,183,198,195,183,180,175,185,185,Norma Dumont,-225,-225,-220,-235,-240,-240,-235,-222,-225,-225,-220,-112,+112,"[184, -228]",
6,Brandon Royval,-240,-275,-260,-265,-255,-270,-265,-250,-250,-245,-220,Matt Schnell,195,220,210,205,210,215,205,200,200,205,185,-124,+124,"[-254, 205]",
7,Blagoy Ivanov,-145,-145,-155,-152,-156,-150,-152,-152,-150,-150,-151,Marcos Rogerio de lima,125,120,130,122,132,125,122,125,120,130,131,,,,
8,Danny Roberts,-105,-110,-105,-103,-104,100,-103,100,-105,-105,-110,Francisco Trinaldo,-115,-110,-115,-121,-112,-120,-121,-125,-120,-115,-110,-101,+101,"[-68, -117]",
9,Melissa Gatto,125,120,120,128,128,150,128,130,120,130,120,Tracy Cortez,-145,-145,-140,-159,-152,-180,-159,-161,-150,-150,-140,-104,+104,"[127, -153]",
